In [24]:
#FIRST of all, install SSMS (SQL Server MicroSoft)
#Second of all, open the SQLQuery1.sql file in SqlServer
#Thrid of all, highlight ONLY the first command (create database Brokerage) then click Execute
#Fourth of all, highlight the rest of the file lines then click Execute.
#by now you should have the database and schema set and done

#now you will need to populate it with data

#Fifth of all, install pyodbc (nice library btw)

#you will then want to connect your database to the ODBC:
#   1) open SQL Server
#   2) right click on the very first element in object explorer (which is your device obvs)
#   3) choose connect
#   4) copy the server name
#   5) paste it in the server string below


#Sixth of all, just run everything below and enjoy :)

#OPTIONAL : you don't need to know how to generate the report file, it is so ez though
#   just open SSMS
#   Select the database "Brokerage"
#   Right click and select Reports then Standard Reports
#   Select any of the reports than seems interesting

In [ ]:
import pandas as pd
import pyodbc
import random

In [60]:
df = pd.read_csv('MOCK_DATA.csv')
#Columns in the CSV : first_name Minit last_name Address gender BDate Salary
#attributes in our database: Fname Minit Lname Bdate Address_ Sex Salary
#print(df['first_name'])
alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

#for i in range(1000):
#    df.loc[i,'Minit'] = random.choice(alphabet)
#df.to_csv('MOCK_DATA.csv')

#print(df.iloc[:,:].values.tolist())


In [68]:
names_df = pd.read_csv('NationalNames.csv')
Address_df = pd.read_csv('Street_Address_Listing.csv')
while (len(Address_df) < 1000000):
    Address_df.append(Address_df)

for i in range(100000):
    df.loc[i, 'first_name'] = names_df.loc[i, 'Name']
    df.loc[i, 'Minit'] = random.choice(alphabet)
    df.loc[i, 'last_name'] = names_df.loc[i, 'Name']
    df.loc[i, 'Address'] = Address_df.loc[i, 'FULL ADDRESS']
    randDay = random.choice(range(1,29))
    randMonth = random.choice(range(1,12))
    randYear = random.choice(range(2000,2023))
    randSex = random.choice(range(0,3))
    randSalary = random.choice(range(1, 100000))
    Random_Date = pd.to_datetime(str(randDay) + "/"+str(randMonth)+"/"+str(randYear))
    df.loc[i, 'BDate'] = Random_Date
    df.loc[i, 'Sex'] = randSex
    df.loc[i, 'Salary'] = randSalary

df.to_csv('another_MOCK_DATA.csv')

KeyboardInterrupt: 

In [ ]:
Department_df = pd.read_csv('2019_free_title_data')
while(len(Department_df)<1000000):
    Department_df.append(Department_df)
for i in range(100000):
    

In [61]:

server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()


insert_to_employee = "INSERT INTO Employee (Fname, Minit, Lname, Bdate, Address_, Sex, Salary) VALUES(?,?,?, ?, ?, ?, ?)"
select_all_from_employee = "select * from Employee"



for i in range(1000):
    #lst = [df['first_name'], df['Minit'], df['last_name'], pd.to_datetime(df['BDate']), df['Address'], df['gender'].astype(int), df['Salary'].astype(int)]
    lst = df.iloc[:,:].values.tolist()
    cursor.executemany(insert_to_employee, lst)

connection.commit()




In [66]:
get_Employees_count = "select count (*) from Employee"

cursor = connection.cursor()
count = cursor.execute(get_Employees_count)
print(list(count))
#for row in cursor:
#    print(row)

Error: ('HY000', '[HY000] [Microsoft][ODBC Driver 17 for SQL Server]Connection is busy with results for another command (0) (SQLExecDirectW)')

In [11]:
for driver in pyodbc.drivers():
    print(driver)

SQL Server
SQL Server Native Client 11.0
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server
Microsoft Access Driver (*.mdb, *.accdb)
Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
Microsoft Access Text Driver (*.txt, *.csv)


In [22]:

lst = [df['first_name'][0], df['Minit'][0], df['last_name'][0], pd.to_datetime(df['BDate'][0]), df['Address'][0], int(df['gender'][0]), int(df['Salary'][0])]

server = 'LAPTOP-66BOPJMV\SQLEXPRESS'
database = 'Brokerage_Firm'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            server='+server+';\
                            database='+database+';\
                            trusted_connection=yes;')
cursor = connection.cursor()
cursor.execute("INSERT INTO Employee (Fname, Minit, Lname, Bdate, Address_, Sex, Salary) VALUES(?,?,?, ?, ?, ?, ?)", lst)